In [158]:
import pandas as pd
import datetime
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [159]:
df = pd.read_csv('../data/show_no_show.csv', index_col='Unnamed: 0')

In [160]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,None,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,None,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,None,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,None,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,None,0,0,No


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110182 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  object 
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(7), object(6)
memory usage: 12.6+ MB


In [162]:
df.dropna(inplace=True)

# ScheduledDay & AppointmentDay into datetime

In [163]:
def to_datetime(ex):
    return datetime.datetime(int(ex[0:4]), int(ex[5:7]), int(ex[8:10]), int(ex[11:13]), int(ex[14:16]), int(ex[17:19]))

In [164]:
df['ScheduledDay_hours'] = df['ScheduledDay']

In [165]:
df['ScheduledDay']=df['ScheduledDay'].apply(lambda ex: datetime.date(int(ex[0:4]), int(ex[5:7]), int(ex[8:10])))

In [166]:
df['ScheduledDay_hours']=df['ScheduledDay_hours'].apply(lambda ex: int(ex[11:13]))

In [167]:
df['AppointmentDay']=df['AppointmentDay'].apply(lambda ex: datetime.date(int(ex[0:4]), int(ex[5:7]), int(ex[8:10])))

In [168]:
df['DaysDifference']= df['AppointmentDay']-df['ScheduledDay']

In [169]:
df['DaysDifference_Num'] = df['DaysDifference'].dt.days.astype('int16')

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110182 entries, 0 to 110526
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   PatientId           110182 non-null  float64        
 1   AppointmentID       110182 non-null  int64          
 2   Gender              110182 non-null  object         
 3   ScheduledDay        110182 non-null  object         
 4   AppointmentDay      110182 non-null  object         
 5   Age                 110182 non-null  int64          
 6   Neighbourhood       110182 non-null  object         
 7   Scholarship         110182 non-null  int64          
 8   Hipertension        110182 non-null  int64          
 9   Diabetes            110182 non-null  int64          
 10  Alcoholism          110182 non-null  object         
 11  Handcap             110182 non-null  int64          
 12  SMS_received        110182 non-null  int64          
 13  No-show       

In [171]:
df.tail()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledDay_hours,DaysDifference,DaysDifference_Num
110522,2.572134e+12,5651768,F,2016-05-03,2016-06-07,56,MARIA ORTIZ,0,0,0,None,0,1,No,9,35 days,35
110523,3.596266e+12,5650093,F,2016-05-03,2016-06-07,51,MARIA ORTIZ,0,0,0,None,0,1,No,7,35 days,35
110524,1.557663e+13,5630692,F,2016-04-27,2016-06-07,21,MARIA ORTIZ,0,0,0,None,0,1,No,16,41 days,41
110525,9.213493e+13,5630323,F,2016-04-27,2016-06-07,38,MARIA ORTIZ,0,0,0,None,0,1,No,15,41 days,41
110526,3.775115e+14,5629448,F,2016-04-27,2016-06-07,54,MARIA ORTIZ,0,0,0,None,0,1,No,13,41 days,41


# Gender into category

In [172]:
df["Alcoholism_Num"] = df["Alcoholism"].replace("None", 0).replace("Low",1).replace("Moderate", 2).replace("High", 3)

In [173]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'ScheduledDay_hours', 'DaysDifference', 'DaysDifference_Num',
       'Alcoholism_Num'],
      dtype='object')

In [174]:
df["No-show"].value_counts()

No     87951
Yes    22231
Name: No-show, dtype: int64

In [175]:
df["Attendance"] = df["No-show"].replace("No", 1).replace("Yes", 0) 

In [176]:
df["No-show_Num"] = df["No-show"].replace("No", 0).replace("Yes", 1) 

In [177]:
df["Gender_Num"] = df["Gender"].replace("M", 0).replace("F", 1) 

In [178]:
df["Neighbourhood"].value_counts()

JARDIM CAMBURI                 7698
MARIA ORTIZ                    5795
RESISTÊNCIA                    4411
JARDIM DA PENHA                3865
ITARARÉ                        3500
                               ... 
ILHA DO BOI                      35
ILHA DO FRADE                    10
AEROPORTO                         8
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: Neighbourhood, Length: 81, dtype: int64

In [189]:
df_mean = df[["Neighbourhood","No-show_Num"]].groupby("Neighbourhood").mean("No-show_Num")

In [190]:
df = df.join(df_mean, on = "Neighbourhood", rsuffix = "_Neighbourhood", how = "left")

In [191]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,...,No-show,ScheduledDay_hours,DaysDifference,DaysDifference_Num,Alcoholism_Num,Attendance,No-show_Num,Gender_Num,No-show_Num_Neighbourhood,No-show_Num_Neighbourhood
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,...,No,18,0 days,0,0,1,0,1,0.163001,0.163001
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,...,No,16,0 days,0,0,1,0,0,0.163001,0.163001
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,...,No,16,0 days,0,0,1,0,1,0.169782,0.169782
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,...,No,17,0 days,0,0,1,0,1,0.176471,0.176471
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,...,No,16,0 days,0,0,1,0,1,0.163001,0.163001


In [192]:
df.columns

Index(['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay',
       'AppointmentDay', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'No-show',
       'ScheduledDay_hours', 'DaysDifference', 'DaysDifference_Num',
       'Alcoholism_Num', 'Attendance', 'No-show_Num', 'Gender_Num',
       'No-show_Num_Neighbourhood', 'No-show_Num_Neighbourhood'],
      dtype='object')

In [193]:
X = df[['Gender_Num', 'Age', 'Scholarship', 'Hipertension','Diabetes', 'Handcap', 'SMS_received', 'ScheduledDay_hours', 'DaysDifference_Num', 'Alcoholism_Num', 'No-show_Num_Neighbourhood']]

In [194]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110182 entries, 0 to 110526
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Gender_Num                 110182 non-null  int64  
 1   Age                        110182 non-null  int64  
 2   Scholarship                110182 non-null  int64  
 3   Hipertension               110182 non-null  int64  
 4   Diabetes                   110182 non-null  int64  
 5   Handcap                    110182 non-null  int64  
 6   SMS_received               110182 non-null  int64  
 7   ScheduledDay_hours         110182 non-null  int64  
 8   DaysDifference_Num         110182 non-null  int16  
 9   Alcoholism_Num             110182 non-null  int64  
 10  No-show_Num_Neighbourhood  110182 non-null  float64
 11  No-show_Num_Neighbourhood  110182 non-null  float64
dtypes: float64(2), int16(1), int64(9)
memory usage: 10.3 MB


In [195]:
VIF = [variance_inflation_factor(X, idx) for idx in range(10)]

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [117]:
help(variance_inflation_factor)

Help on function variance_inflation_factor in module statsmodels.stats.outliers_influence:

variance_inflation_factor(exog, exog_idx)
    variance inflation factor, VIF, for one exogenous variable
    
    The variance inflation factor is a measure for the increase of the
    variance of the parameter estimates if an additional variable, given by
    exog_idx is added to the linear regression. It is a measure for
    multicollinearity of the design matrix, exog.
    
    One recommendation is that if VIF is greater than 5, then the explanatory
    variable given by exog_idx is highly collinear with the other explanatory
    variables, and the parameter estimates will have large standard errors
    because of this.
    
    Parameters
    ----------
    exog : ndarray
        design matrix with all explanatory variables, as for example used in
        regression
    exog_idx : int
        index of the exogenous variable in the columns of exog
    
    Returns
    -------
    vif : float